# Aplicaciones de Python

En esta libreta veremos el uso de Google Earth Engine mediante la API de Python. Earth Engine es una plataforma de Google que es considerada la base de datos más grande del planeta, la cual contiene información geoespacial de diversos tipos: datos vectoriales e imágenes y productos satelitales en su mayoría.

Para una breve introducción a Earth Engine revisar mi post [aquí](https://vilcagamarracf.github.io/blog/post4_ee.html).

## 1. Instalación

Es recomendable siempre usar la documentación de cualquier librería ya que los mismos desarrolladores resumen de forma concisa el funcionamiento de su producto. Por ello revisar [Python Installation - EE](https://developers.google.com/earth-engine/guides/python_install). Aquí se explica su instalación (con ejemplo en Colab) y algunas diferencias entre usar Python o JavaScript como lenguaje de programación para desarrollo.

Para su instalación: 
- Usando el terminal de Anaconda 
```
  $ conda install -c conda-forge earthengine-api
```

- Usando PIP (en Colab es mucho más accesible trabajar con Earth Engine aunque tiene sus ventajas y desventajas)

```
    # Para instalar
    $ pip install earthengine-api 

    # Para actualizar la librería (se actualiza mucho lo cual es bueno)
    $ pip install earthengine-api --upgrade
```

Nota: el símbolo `$` representa el >> (o prompt) de un terminal. 
- En caso de usar en Colab, el `$` se usa como signo de exclamación `!`.
- En el caso de Jupyter Notebooks usar un `%` en lugar del `$`.

En mi caso ya tengo instalado earth engine por lo cual solo actualizaré la librería:
- Versión inicial: `earthengine-api-0.1.243`
- Versión actualizada (al momento): `earthengine-api-0.1.253`

In [2]:
# pip install earthengine-api
%pip install earthengine-api --upgrade

  Using cached httplib2shim-0.0.3-py2.py3-none-any.whl
  Created wheel for earthengine-api: filename=earthengine_api-0.1.253-py3-none-any.whl size=178056 sha256=98ae7f077b0c70a7f54b36535ddf980ca489664baf6784b70665551c7a1ee342
  Stored in directory: c:\users\lenovo\appdata\local\pip\cache\wheels\61\06\3f\a581b7796d4c930e05cbbe70ebddcd792298dcfbb964f74ddb
Successfully built earthengine-api
  Attempting uninstall: earthengine-api
    Found existing installation: earthengine-api 0.1.243
    Uninstalling earthengine-api-0.1.243:
      Successfully uninstalled earthengine-api-0.1.243
Note: you may need to restart the kernel to use updated packages.


## 2. Inicio

El nombre de la librería instalada es `earthengine-api` pero se importa como `ee` (cosas de desarrollo).

In [3]:
# Versión de librería
import ee
ee.__version__

'0.1.253'

Para comenzar es requisito haberse registrado en la plataforma y haber activado la cuenta.

In [9]:
ee.Initialize() # Activa un while loop en el cual debes introducir un código (según cuenta) y dar enter

Antes de comenzar hablemos de comparaciones (algunos puntos y opinión personal extra)

### Comparaciones entre el uso en JavaScript y Python

-----

**Usando JavaScript**

Ventajas: \
Comparando con el editor web que usa JavaScript en su plataforma (editor code), éste posee ya integrado: 

- un mapa para visualizar, widgets interactivos que permiten la edición (creación de puntos, figuras, líneas) 
- un panel donde es posible administrar archivos, leer documentación y ejercicios, 
- y otro panel capaz de mostrar resultados y gráficas. 

Desventajas:
- Trabaja como script (corre todo el código "de porrazo")
- **es agobiante escribir código ahí.**

-----

**Usando Python**

Ventajas:
- Mientras que en Python permite más libertad ya que es adaptable a cualquier editor (VS Code, VIM, **sublime text no xd**) o IDE (Spyder, PyCharm, JupyterNotebooks **dependiendo de la experticia del usuario para adaptarlos a su ambiente de desarrollo**).
- Permite trabajar tanto en modo script como libreta jupyter (lo cual permite desarrollar proyectos enteros)  
- y la interacción con un sin fin de librerías del ambiente de Python.

Desventaja (posible ventaja ya que es configurable al gusto):
- Comenzar un proyecto desde cero (literal). 
- Adaptar un widget (interactividad) es posible en Jupyter Notebooks y no en Colab pero requiere conocimiento para adaptarlo.

### 3. Librerías previas

Recordar que estamos comenzando desde cero (no hay nada, todo está en blanco). Para agregar un panel de visualización interactiva tenemos la librería `folium` y también `ipyleaflet` (no corre en Colab por el momento).

Por el momento solo veremos `folium`

1. Importar folium
2. (Configurable a gusto) Definir una función para visualización: mapdisplay()
3. Agregar parámetros de visualización (tanto como interactivos) a gusto. Revisar [Ejemplos de Folium](https://nbviewer.jupyter.org/github/python-visualization/folium/tree/master/examples/)

In [34]:
# Librerías previas

# Función mapdisplay
import folium 
def mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    Crea mapas interactivos usando la librería `folium`

    - `center`: Center of the map (Latitude and Longitude) (`list`)
    - `dicc`: Earth Engine Geometries or Tiles dictionary (`dict`)
    - `Tiles=`: 
    Mapbox Bright 
    Mapbox Control Room
    Stamen Terrain
    Stamen Toner
    stamenwatercolor
    cartodbpositron
    - `zoom_start=`: Initial zoom level for the map (`int`)

    Return: A folium.Map object
    '''
    
    # Centrado de imagen: Admite una lista que contiene las coordenadas geográficas en forma decimal
    # Ejemplo: [-76.0908, -13.4589], donde -76 sería longitud (x) y -13 latitud (y)
    center = center[::-1] # El [::-1] lee inversamente la lista ingresada según Earth Engine

    # Parámetros de visualización de función mapdisplay
        
    global mapViz # Para leer la imagen posteriormente mediante variable mapViz
    mapViz = folium.Map(location=center,
                        tiles=Tiles, 
                        zoom_start=zoom_start, 
                        control_scale=True)
    
    # Leer imágenes de Earth Engine y agregar a Folium
    for k,v in dicc.items():
        if ee.image.Image in [type(x) for x in v.values()]:
            folium.TileLayer(tiles = v["tile_fetcher"].url_format,
                             attr  = 'Google Earth Engine',
                             overlay =True,
                             name  = k).add_to(mapViz)
        else:
            folium.GeoJson(data = v,
                           name = k).add_to(mapViz)
    
    
    # Agregando cosas: https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/MiniMap.ipynb
    # Minimapa
    from folium.plugins import MiniMap
    minimap = MiniMap(
        tile_layer="OpensTreetMap", 
        position="bottomright", 
        zoom_level_fixed=7)
    mapViz.add_child(minimap) 
    
    # Agregar: botón Fullscreen
    from folium.plugins import Fullscreen
    Fullscreen().add_to(mapViz)

    # Botón de control de Capas (Layers)
    mapViz.add_child(folium.LayerControl())
    
    return mapViz

print('mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10) listo!')

mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10) listo!


Crea mapas interactivos usando la librería `folium`, función `mapdisplay()`:

```
                            mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
```
donde:
- `center`: Centrado de imagen (Latitud y Longitud) (`list`)
- `dicc`: Geometrías de Earth Engine o Diccionario con capas (`dict`)
- `Tiles=`: Mapas base
    - Mapbox Bright 
    - Mapbox Control Room
    - Stamen Terrain
    - Stamen Toner
    - stamenwatercolor
    - cartodbpositron
- `zoom_start=`: Nivel de zoom inicial para visualización (`int`)

### 4. Ejemplo

#### Región de Interés (ROI): Zona de Chincha

He dibujado la región del Valle de chincha en Arcgis y subido a mi Asset en Earth Engine. El ID de uso (personal) es `'users/CesarVilca/area_total'`.

- Asignamos una lista con las coordenadas para centrado.
- Creamos un diccionario con las "capas" para visualizar.
- Uso de la función `mapdisplay` para visualización con zoom de 11.

In [36]:
# ROI CHINCHA 

area_total = ee.FeatureCollection('users/CesarVilca/area_total') 
roi_clip = area_total.geometry().bounds()  # Representación de un cuadrado representando los límites

# coordenadas de centrado
centroide = [-76.0908, -13.4589]

# Diccionario de visualización
dicc = {
    'area total': area_total.getInfo(), 
    'roi_clip'  : roi_clip.getInfo()
}

# función de visualización
mapdisplay(centroide, dicc, zoom_start=11)

#### Funciones para visualizar fechas

In [38]:
# ver_rangos

Start_date = '2012-01-01'
End_date   = '2021-01-31'
range_date = [Start_date, End_date]

def ver_rangos(img_col, range_date= [Start_date, End_date]):
    """
     Devuelve los rangos de fechas existente en un rango inicial escrito manualmente como input.
     Pasos:
     1. `Reducer.minMax()` devuelve un valor min y max
     2. `icol.reduceColumns()` devuelve un diccionario con el min-max
     more info: https://developers.google.com/earth-engine/apidocs/ee-imagecollection-reducecolumns?hl=en
    """

    rango = img_col.reduceColumns(ee.Reducer.minMax(), ["system:time_start"]) 
    # Retorna un ee.dictionary: {'max': 1608132402761, 'min': 1545060401000}

    # Obtención de la fecha min y max del image collection en formato ISO standard 8601
    # Javascript trabaja las fechas con milisegundos (se deja así)
    # Python     trabaja las fechas con segundos (por eso /1000)
    init_date = ee.Date(rango.get('min')).getInfo()['value']/1000.
    last_date = ee.Date(rango.get('max')).getInfo()['value']/1000.

    # Dar formato a las fechas  
    from datetime import datetime as dt
    init_date_f = dt.utcfromtimestamp(init_date).strftime('%Y-%m-%d') # %H:%M:%S
    last_date_f = dt.utcfromtimestamp(last_date).strftime('%Y-%m-%d') # %H:%M:%S

    # Obtenga el rango de fechas de las imágenes en la colección.
    range_date = [Start_date, End_date]
    print('Total imágenes: {}'.format(img_col.size().getInfo()))
    print('Rango Temporal Ingresado : {} - {}'.format(range_date[0], range_date[1]))
    print('Rango Temporal Real      : {} - {}'.format(init_date_f, last_date_f))
    
print('ver_rangos(icol, range_date= [Start_date, End_date]) listo!')

ver_rangos(icol, range_date= [Start_date, End_date]) listo!


In [39]:
# conv_fecha 

def conv_fecha(img, getInfo=None):
    '''
      Convierte el formato de UNIX Time al formato %Y-%m-%d %H:%M:%S
      img: ee.Image
      getInfo = True : cuando se tiene ya el .getInfo() 
    '''
    from datetime import datetime as dt
    if getInfo is None:
        return dt.utcfromtimestamp(img.getInfo()['properties']["system:time_start"]/1000).strftime('%Y-%m-%d %H:%M:%S')
    if getInfo==True:
        return dt.utcfromtimestamp(img['properties']["system:time_start"]/1000).strftime('%Y-%m-%d %H:%M:%S')
    
print('conv_fecha(img, getInfo=None) listo!')

conv_fecha(img, getInfo=None) listo!


#### Visualización rápida Sentinel-2

Primera imagen registrada: `2018-12-17`

In [43]:
# 1. Seleccionamos las fechas de interés:
Start_date = '2012-12-17'  
End_date   = '2021-02-11'  
range_date = [Start_date, End_date]

# 2. Filtrado de imágenes Sentinel-2
img_s2 = ee.ImageCollection('COPERNICUS/S2_SR')\
        .filterDate(Start_date, End_date)\
        .filterBounds(ee.Geometry.Point(-76.13, -13.48))\
        .sort('CLOUDY_PIXEL_PERCENTAGE')

img_s2_rgb = img_s2.first().multiply(0.0001) # .clip(roi_clip)

# Fecha de toma
ver_rangos(img_s2, range_date=range_date)

# ID
print()
print('Mejor imagen:')
print('Nubosidad: {a}%'.format(a = img_s2.first().getInfo()['properties']['CLOUDY_PIXEL_PERCENTAGE']))
print('Image ID : {ID}'.format(ID = img_s2.first().getInfo().get('id')))
print('Fecha    : {Fecha}'.format(Fecha = conv_fecha(img_s2.first())))

# Visualizacion: Mapdisplay
bands = ['B4','B3','B2']
dict_img_s2 = {'img_s2': img_s2_rgb.getMapId({'min':0.0, 'max':0.3, 'bands': bands})}
mapdisplay(centroide, dict_img_s2, zoom_start=12)

Total imágenes: 156
Rango Temporal Ingresado : 2012-12-17 - 2021-02-11
Rango Temporal Real      : 2018-12-17 - 2021-02-09

Mejor imagen:
Nubosidad: 0.200305%
Image ID : COPERNICUS/S2_SR/20190501T151711_20190501T151806_T18LUL
Fecha    : 2019-05-01 15:26:50


### Conclusiones

Earth Engine es un mundo ya que posee más de 800 funciones (según su artículo científico [aquí](https://www.sciencedirect.com/science/article/pii/S0034425717302900)) pero brinda un poder computacional enorme para trabajar las imágenes.